## 8. Model Deployment Using FastAPI

### Import Necessary Libraries

In [12]:
import wandb
import os
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel, Field
import joblib
import pandas as pd
import uvicorn

In [13]:
ENTITY = "nisarg-patel2815-jio-institute"
PROJECT = "MLOps_Adult_Income"

api = wandb.Api()

# Fetch all runs in the project
runs = api.runs(f"{ENTITY}/{PROJECT}")

# Find the run with the highest Test Accuracy
best_run = max(runs, key=lambda run: run.summary.get("Test Accuracy", 0))

# Fetch the best model artifact from the best run
artifacts = [artifact for artifact in best_run.logged_artifacts()]

if artifacts:
    best_model_artifact = artifacts[0]  # Assuming first artifact is the model
    best_model_path = best_model_artifact.download()
    print(f"✅ Best model with highest Test Accuracy downloaded to: {best_model_path}")
else:
    print("⚠️ No model artifacts found in the best run.")


wandb:   1 of 1 files downloaded.  


✅ Best model with highest Test Accuracy downloaded to: /Users/nisarg/Documents/JIO INSTITUTE/Quarter4/ML Engineering/mlops-adult-income/artifacts/Gradient_Boosting:v0


In [ ]:
# Apply nest_asyncio to allow FastAPI to run inside Jupyter
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Define path to downloaded model
if "model" not in globals():
    model_file = None
    for root, _, files in os.walk(best_model_path):
        for file in files:
            if file.endswith(".pkl") or file.endswith(".joblib"):
                model_file = os.path.join(root, file)
                break

    if model_file is None:
        raise Exception("⚠️ No valid model file found in the downloaded artifacts.")

    # Load the model dynamically
    model = joblib.load(model_file)
    print(f"✅ Model loaded from: {model_file}")
else:
    print("✅ Model already loaded, skipping reloading.")

# Define input data schema using Pydantic
class InputData(BaseModel):
    age: int = Field(..., ge=17, description="Age must be at least 17")
    workclass: str
    fnlwgt: int
    education: str
    education_num: int = Field(..., ge=1, le=16, description="Education num must be between 1 and 16")
    marital_status: str
    occupation: str
    relationship: str
    race: str
    sex: str
    capital_gain: int = Field(..., ge=0, description="Capital gain must be ≥ 0")
    capital_loss: int = Field(..., ge=0, description="Capital loss must be ≥ 0")
    hours_per_week: int = Field(..., ge=1, le=99, description="Hours per week must be between 1 and 99")
    native_country: str

# Define mapping for model prediction output
income_mapping = {0: "<=50K", 1: ">50K"}

@app.get("/")
def home():
    return {"message": "Welcome to the FastAPI Income Prediction API! Use /predict to get predictions."}

# Prediction endpoint
@app.post("/predict")
def predict(data: InputData):
    try:
        # Convert input data to a DataFrame
        input_df = pd.DataFrame([data.model_dump()])
        
        # Make prediction
        prediction = model.predict(input_df)

        # Check if prediction is already a string
        if isinstance(prediction[0], str):
            predicted_income = prediction[0]  # Use as is if already a string
        else:
            predicted_income = income_mapping[prediction[0]]  # Convert numerical prediction to label

        return {"predicted_income": predicted_income}

    except Exception as e:
        return {"error": str(e)}

# Run FastAPI inside Jupyter Notebook
uvicorn.run(app, host="127.0.0.1", port=8000)
